# openAI API key 받기
- https://platform.openai.com/api-keys 접속 후 로그인
- create new secret key 진행
- 생성된 secret key 복사 (요금이 청구되므로 유출되지 않도록 주의! 유출되었다면 기존 키를 폐기하고 새로운 키를 받을 것)
- https://platform.openai.com/settings/organization/billing/overview 에 들어가 가격을 결제할 것

- 자세한 사용법은 https://github.com/openai/openai-python 참조

In [ ]:
# !pip install openai

In [16]:
from openai import OpenAI
import openai
import os


api_key = "your api key here"
client = openai.OpenAI(api_key=api_key)

### Prompt engineering

In [10]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "I am a Ph.D. in physics"},
        {"role": "user", "content": "Why is the ice slippery?"}
    ],
    stream=True)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

The slipperiness of ice is primarily due to the thin layer of liquid water that forms on its surface. When pressure is applied, such as the weight of a person's foot, the ice can melt slightly, creating a thin layer of water. This liquid water acts as a lubricant, reducing the friction between the ice and the object moving across it, making it slippery. This phenomenon is known as "pressure melting" and is what causes ice to be slippery even when temperatures are below freezing.

In [11]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role" : "system", "content" : "I am five-years old"},
        {"role" : "user", "content" : "Why is the ice slippery?"}
    ],
    stream=True)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

Ice is slippery because when you step on it, the pressure from your weight melts the top layer of the ice just a little bit. This creates a thin layer of water on the surface, which makes it slippery for you to walk on. It's like how a banana peel can make a floor slippery because it releases a thin layer of oil when you step on it.

In [13]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role" : "system", "content" : "I am a Ph.D. in physics"},
        {"role" : "user", "content" : "Why is the ice slippery?"},
        {"role" : "assistant", "content" : "We are learning about friction force."}
    ],
    stream=True)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

The slipperiness of ice can be attributed to the thin layer of water that forms on its surface. When you step on ice, the pressure exerted by your weight can cause a small amount of ice to melt, creating a thin layer of water between the ice and your foot. This thin layer of water significantly reduces the friction between your foot and the ice, making it slippery. The low friction coefficient of ice combined with this thin layer of water results in the slippery nature of ice.

### RAG

In [8]:
question = "There is Jack, John, and Jane. Who is the murderer?"
search_result = search_documents(question)

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role" : "system", "content" : "You are a detective"},
        {"role" : "user", "content" : question}
    ],
    stream=True)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

I would need more information to determine who the murderer is. Can you provide me with some details about the case or the events leading up to the murder?

In [9]:
retrieve = {"Jack": "He is the murderer.",
            "John": "He is innocent.",
            "Jane": "She is innocent."}

def search_documents(question):
    for key in retrieve:
        if key.lower() in question.lower():
            return retrieve[key]
    return "No relevant information found."

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a detective"},
        {"role": "user", "content": question},
        {"role": "system", "content": "Search result: {}".format(search_result)}
    ],
    stream=True
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

Based on the information I found, it appears that Jack is the murderer. Is there anything else you would like to know or discuss about this case?

### Fine-tuning
- https://platform.openai.com/docs/guides/fine-tuning

In [50]:
import json
from pathlib import Path

# 최소 10개의 메시지 필요
# 메시지 형식 준수

data = [
    {
        "messages": [
            {"role": "system", "content": "You are a football scout."},
            {"role": "user", "content": "Among Ronaldo, Messi, and Neymar, who is the best?"},
            {"role": "assistant", "content": "Messi"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a football scout."},
            {"role": "user", "content": "Among Ronaldo, Neymar, and Benzema, who is the best?"},
            {"role": "assistant", "content": "Ronaldo"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a football scout."},
            {"role": "user", "content": "Among Neymar, Benzema, and Lewandowski, who is the best?"},
            {"role": "assistant", "content": "Neymar"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a football scout."},
            {"role": "user", "content": "Among Benzema, Kane, and Lewandowski, who is the best?"},
            {"role": "assistant", "content": "Lewandowski"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a football scout."},
            {"role": "user", "content": "Among Benzema, Kane, and Suarez, who is the best?"},
            {"role": "assistant", "content": "Benzema"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a football scout."},
            {"role": "user", "content": "Among Modric, Kane, and Suarez, who is the best?"},
            {"role": "assistant", "content": "Modric"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a football scout."},
            {"role": "user", "content": "Among Henry, Kane, and Suarez, who is the best?"},
            {"role": "assistant", "content": "Henry"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a football scout."},
            {"role": "user", "content": "Among Bale, Kane, and Suarez, who is the best?"},
            {"role": "assistant", "content": "Suarez"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a football scout."},
            {"role": "user", "content": "Among Bale, Kane, and Salah, who is the best?"},
            {"role": "assistant", "content": "Bale"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a football scout."},
            {"role": "user", "content": "Among Son, Kane, and Salah, who is the best?"},
            {"role": "assistant", "content": "Kane"}
        ]
    }
]

with open('finetuning_data.jsonl', 'w', encoding='utf-8-sig') as json_file:
    for item in data:
        json_file.write(json.dumps(item) + "\n")

In [51]:
uploaded_file = client.files.create(file=open("finetuning_data.jsonl", "rb"), purpose="fine-tune")
file_id = uploaded_file.id
file_id

'file-TMgDqtMwOBTQrlN58rDnhADG'

In [52]:
fine_tune_job = client.fine_tuning.jobs.create(training_file=file_id, model="gpt-3.5-turbo")
fine_tune_job_id = fine_tune_job.id
events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=fine_tune_job_id, limit=10)
for event in events.data:
    print(event)

FineTuningJobEvent(id='ftevent-2Zkk8RbQzI91FvO9Qvp2jiZY', created_at=1725400041, level='info', message='Validating training file: file-TMgDqtMwOBTQrlN58rDnhADG', object='fine_tuning.job.event', data={}, type='message')
FineTuningJobEvent(id='ftevent-RWX5Hqn5lVUSoadnRYsh2NvD', created_at=1725400041, level='info', message='Created fine-tuning job: ftjob-aNJfQdQrVEpjKhtXOskHc5CG', object='fine_tuning.job.event', data={}, type='message')


In [57]:
# 업로드 상태 확인
fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tune_job_id)
print(fine_tune_job.status)  # 'succeeded' 또는 'failed'
fine_tuned_model_id = fine_tune_job.fine_tuned_model  # 파인튜닝된 모델의 ID

running


In [58]:
events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=fine_tune_job_id, limit=10)
for event in events.data:
    print(event)

FineTuningJobEvent(id='ftevent-jKUqPRTrfgVpx13R9Ag75P52', created_at=1725402023, level='info', message='Step 37/100: training loss=0.00', object='fine_tuning.job.event', data={'step': 37, 'train_loss': 0.00246429443359375, 'total_steps': 100, 'train_mean_token_accuracy': 1.0}, type='metrics')
FineTuningJobEvent(id='ftevent-W0UscZoEORXDRbIqQH2k8wse', created_at=1725402019, level='info', message='Step 36/100: training loss=0.93', object='fine_tuning.job.event', data={'step': 36, 'train_loss': 0.926629364490509, 'total_steps': 100, 'train_mean_token_accuracy': 0.8333333134651184}, type='metrics')
FineTuningJobEvent(id='ftevent-22lHYNe9itDJBlRx9jL5Hlqp', created_at=1725402019, level='info', message='Step 35/100: training loss=1.47', object='fine_tuning.job.event', data={'step': 35, 'train_loss': 1.466423511505127, 'total_steps': 100, 'train_mean_token_accuracy': 0.75}, type='metrics')
FineTuningJobEvent(id='ftevent-Vlv8037aCrJIA2JkUtN3Yen0', created_at=1725402019, level='info', message='St

In [63]:
stream = client.chat.completions.create(
    model= "ft:gpt-3.5-turbo-0125:personal::A3WDgJPH",
    messages=[
        {"role" : "system", "content" : "You are a football scout."},
        {"role" : "user", "content" : "Among Messi, Maradona, and Pele, who is the best?"}
    ],
    stream=True)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

Messi

In [66]:
import pickle

finetuned_model = "ft:gpt-3.5-turbo-0125:personal::A3WDgJPH"
with open(file='finetuned_model.pickle', mode='wb') as f:
    pickle.dump(finetuned_model, f)

In [73]:
stream = client.chat.completions.create(
    model= "ft:gpt-3.5-turbo-0125:personal::A3WDgJPH",
    messages=[
        {"role" : "system", "content" : "You are a football scout."},
        {"role" : "user", "content" : "Among Son, Salah, and Kane, who is the best?"}
    ],
    stream=True)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

Kane